In [ ]:
import plotly.express as px
import pandas as pd
import unidecode
import datetime
import numpy as np
import seaborn as sn
import folium
import json
from branca.colormap import linear
from IPython.display import display
from folium.plugins import HeatMapWithTime

In [ ]:
df_covid = pd.read_excel("../input/road-transport-brazil/covid-19-05-2020.xlsx",sep =";")

df_roads = pd.read_csv("../input/road-transport-brazil/road-transport-brazil.csv",sep =";")

# Análise da Base Covid  / Covid Data Analysis

Detalhes da base / Data details

In [ ]:
print(df_covid.shape)

print(df_covid.isnull().sum())

In [ ]:
df_covid.describe()

Linhas por região / Line by region

In [ ]:
df_covid.groupby('regiao').count()

Removendo as colunas com altos índices de missing / Removing the column with highlty missing rate

In [ ]:
df_covid.drop(['Recuperadosnovos', 'emAcompanhamentoNovos'], axis=1)

Criando as colunas ratio / Creating ratio columns 

In [ ]:
df_covid['Ratio_casosAcumulado'] = (df_covid['casosAcumulado'] / df_covid['populacaoTCU2019'])*100

df_covid['Ratio_obitosAcumulado'] = (df_covid['obitosAcumulado'] / df_covid['populacaoTCU2019'])*100

Removendo a regiao "Brasil" / Removing the region "Brasil"

In [ ]:
df_covid = df_covid[df_covid["regiao"] != "Brasil"]

Montando a base somente com Estados / Creating a database with States only

In [ ]:
df_covid_estado = df_covid[df_covid["municipio"].isnull() & df_covid["codmun"].isnull()]

Montando a base somente com Município / Creating a database with Cities only

In [ ]:
df_covid_municipio = df_covid[df_covid["municipio"].notnull() & df_covid["codmun"].notnull()]

Agrupando a base por município e estado com a última data de publicação 

In [ ]:
df_covid_municipio['data'] = pd.to_datetime(df_covid_municipio['data'])
df_covid_municipio = df_covid_municipio[df_covid_municipio.groupby('municipio').data.transform('max') == df_covid_municipio['data']]

df_covid_estado['data'] = pd.to_datetime(df_covid_estado['data'])
df_covid_estado = df_covid_estado[df_covid_estado.groupby('estado').data.transform('max') == df_covid_estado['data']]

Removendo os acentos dos municipios / Removing Accents from municipios

In [ ]:
df_covid_municipio['municipio'] =  df_covid_municipio['municipio'].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')

# Análise da Base Roads / Roads Data Analysis

Detalhes da base / Data details

In [ ]:
print(df_roads.shape)

print(df_roads.isnull().sum())

In [ ]:
df_roads.describe()

Removendo as colunas não necessárias / Removing the columns without use

In [ ]:
df_roads = df_roads.drop(['codigo_viagem', 'empresa','nu_linha','onibus','tipo_viagem','sentido_linha','in_transbordo','lat','lon'], axis=1)

Formatando a data_fim_viagem / Formatting the data_fim_viagem

In [ ]:
df_roads['data_fim_viagem'] = pd.to_datetime(df_roads['data_fim_viagem'])

df_roads['data_fim_viagem'] = df_roads['data_fim_viagem'].dt.strftime("%d-%m-%Y")

Filtrando a base para pegar as viagens que ocorrem até a data de estudo da covid / Filtering the base to catch trips that take place until the covid study date

In [ ]:
df_roads = df_roads[(df_roads['data_fim_viagem'] > '01-01-2019' ) & (df_roads['data_fim_viagem'] < '05-05-2019')]

Removendo as colunas não necessárias / Removing the columns without use

In [ ]:
df_roads = df_roads.drop(['origem', 'origem_uf','pdop','data_viagem_programada','data_inicio_viagem'], axis=1)

Criando as bases por estado e municipio com o total de viagens recebidas / Creating the bases by estado and municipio with the total of trips received

In [ ]:
df_roads_municipio = df_roads.drop(['destino_uf','data_fim_viagem'], axis=1)
df_roads_municipio = df_roads_municipio.groupby(['destino']).size().reset_index(name='counts')

df_roads_estado = df_roads.drop(['destino','data_fim_viagem'], axis=1)
df_roads_estado = df_roads_estado.groupby(['destino_uf']).size().reset_index(name='counts')

In [ ]:
df_roads_municipio.rename(columns={'destino':'municipio','counts':'viagens'}, inplace=True)

df_roads_estado.rename(columns={'destino_uf':'estado','counts':'viagens'}, inplace=True)

In [ ]:
df_roads_estado

# Correlações / Correlations

Formantando as palavras chaves para o merge / Forming the keywords for the merge

In [ ]:
df_covid_estado['estado'] = df_covid_estado['estado'].str.upper()
df_roads_estado['estado'] = df_roads_estado['estado'].str.upper()

df_covid_municipio['municipio'] = df_covid_municipio['municipio'].astype(str).str.upper().str.strip()
df_roads_municipio['municipio'] = df_roads_municipio['municipio'].astype(str).str.upper().str.strip()

Unindo as bases para análise da correlação / Joining the bases for correlation analysis

In [ ]:
correlacao_estado = df_roads_estado.merge(df_covid_estado, on='estado', how='left')
correlacao_estado = correlacao_estado[['casosAcumulado','obitosAcumulado','viagens']]

correlacao_municipio = df_roads_municipio.merge(df_covid_municipio, on='municipio', how='left')
correlacao_municipio = correlacao_municipio[['casosAcumulado','obitosAcumulado','viagens']]

In [ ]:
corrMatrix_estado = correlacao_estado.corr()
sn.heatmap(corrMatrix_estado, annot=True)

In [ ]:
corrMatrix_municipio = correlacao_municipio.corr()
sn.heatmap(corrMatrix_municipio, annot=True)

# Mapas / Maps

Mapa do Brasil Casos Acumulados / Brazil Map Acumulated Cases

In [ ]:
br_estados = '../input/brazil-geojson/brazil_geo.json'
geo_json_data = json.load(open(br_estados))

df_covid_estado_dict = df_covid_estado[df_covid_estado["data"] == "2020-05-18"]
df_covid_estado_dict = df_covid_estado_dict.set_index('estado')['casosAcumulado']

colormap = linear.YlOrRd_08.scale(600,64000)

mapa_estados = folium.Map(
    location=[-15.77972, -47.92972], 
    zoom_start=4,
    tiles='cartodbpositron'
)
folium.GeoJson(
    geo_json_data,
    name='2018',
    style_function=lambda feature: {
        'fillColor': colormap(df_covid_estado_dict[feature['id']]),
        'color': 'black',
        'weight': 0.6,
    }
    
).add_to(mapa_estados)


colormap.caption = 'Casos Acumulados de COVID 19 por Estados em 18/05 '
colormap.add_to(mapa_estados)
folium.LayerControl(collapsed=False).add_to(mapa_estados)

mapa_estados

Mapa do Brasil Casos Acumulados (% pop) / Brazil Map Acumulated Cases (% pop)

In [ ]:
br_estados = '../input/brazil-geojson/brazil_geo.json'
geo_json_data = json.load(open(br_estados))

df_covid_estado_dict = df_covid_estado[df_covid_estado["data"] == "2020-05-18"]
df_covid_estado_dict = df_covid_estado_dict.set_index('estado')['Ratio_casosAcumulado']

colormap = linear.YlOrRd_08.scale(0,1)

mapa_estados = folium.Map(
    location=[-15.77972, -47.92972], 
    zoom_start=4,
    tiles='cartodbpositron'
)
folium.GeoJson(
    geo_json_data,
    name='2018',
    style_function=lambda feature: {
        'fillColor': colormap(df_covid_estado_dict[feature['id']]),
        'color': 'black',
        'weight': 0.6,
    }
    
).add_to(mapa_estados)


colormap.caption = 'Casos Acumulados de COVID 19 por Estados em 18/05'
colormap.add_to(mapa_estados)
folium.LayerControl(collapsed=False).add_to(mapa_estados)

mapa_estados

Mapa do Brasil Obitos Acumulados / Brazil Map Acumulated Death

In [ ]:
br_estados = '../input/brazil-geojson/brazil_geo.json'
geo_json_data = json.load(open(br_estados))

df_covid_estado_dict = df_covid_estado[df_covid_estado["data"] == "2020-05-18"]
df_covid_estado_dict = df_covid_estado_dict.set_index('estado')['obitosAcumulado']

colormap = linear.YlOrRd_08.scale(0,5000)

mapa_estados = folium.Map(
    location=[-15.77972, -47.92972], 
    zoom_start=4,
    tiles='cartodbpositron'
)
folium.GeoJson(
    geo_json_data,
    name='2018',
    style_function=lambda feature: {
        'fillColor': colormap(df_covid_estado_dict[feature['id']]),
        'color': 'black',
        'weight': 0.6,
    }
    
).add_to(mapa_estados)


colormap.caption = 'Obitos de COVID 19 por Estados em 18/05 '
colormap.add_to(mapa_estados)
folium.LayerControl(collapsed=False).add_to(mapa_estados)

mapa_estados

Mapa do Brasil Obitos Acumulados (% pop) / Brazil Map Acumulated Death (% pop)

In [ ]:
br_estados = '../input/brazil-geojson/brazil_geo.json'
geo_json_data = json.load(open(br_estados))

df_covid_estado_dict = df_covid_estado[df_covid_estado["data"] == "2020-05-18"]
df_covid_estado_dict = df_covid_estado_dict.set_index('estado')['Ratio_obitosAcumulado']

colormap = linear.YlOrRd_08.scale(0,0.03)

mapa_estados = folium.Map(
    location=[-15.77972, -47.92972], 
    zoom_start=4,
    tiles='cartodbpositron'
)
folium.GeoJson(
    geo_json_data,
    name='2018',
    style_function=lambda feature: {
        'fillColor': colormap(df_covid_estado_dict[feature['id']]),
        'color': 'black',
        'weight': 0.6,
    }
    
).add_to(mapa_estados)


colormap.caption = 'Obitos de COVID 19 por Estados em 18/05'
colormap.add_to(mapa_estados)
folium.LayerControl(collapsed=False).add_to(mapa_estados)

mapa_estados

# Resultados / Results

O objetivo desta análise demonstrar a existência de correlação entre viagens e os casos de COVID-19 no Brazil. Foi contabilizada apenas a quantidade de viagens recebidas no estado/município nos primeiros meses de 2019. Ao analisar a correlação em termos estatísticos ela fica em 0,72 para os estados e 0,79 para os municípios. Ou seja, ambas fortes.
O Brasil não suspendeu atividades de Carnaval e festas e também não impediu de forma eficaz o transporte entre as cidades por um longo período de tempo. É importante destacar a diferença relativa e absoluta de Casos Acumulados e Óbitos Acumulados em cada estado, pois há diferenças significativas conforme demonstrado nos mapas.

The purpose of this analysis is to demonstrate the existence of a correlation between travel and the cases of COVID-19 in Brazil. Only the number of trips received in the state / municipality in the first months of 2019 was counted. When analyzing the correlation in statistical terms, it is 0.72 for the states and 0.79 for the municipalities, both strong.
Brazil has not suspended Carnival and party activities, nor has it effectively prevented transportation between cities for a long period of time. It is important to highlight the relative and absolute difference in Accumulated Cases and Accumulated Deaths in each state, as there are significant differences as shown in the maps.